# [Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)

* Uses [LangGraph persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/) to incorporate `memory`
* Chatbot will be able to have a conversation & remember previous interactions with a [chat model](https://python.langchain.com/docs/concepts/chat_models/)

> ### Setup

In [17]:
import os
from dotenv import load_dotenv

# Load secrets from file
with open('secrets.txt') as f:
    for line in f:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

# Initialize LangChain
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

### Using `ChatModel` directly with `.invoke` method

* `ChatModel` is LangChain "Runnable" →  expose a standard interface for interacting with them.


In [18]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'stop', 'logprobs': None}, id='run-d239188a-9d08-41b1-98d9-4383066d61eb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like to share it, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 11, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d78306d-d00e-4591-a6d2-f2ccd132aaa5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 42, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

* **‼️‼️ Doesn't take the previous conversation turn into context** → This makes for a terrible chatbot experience!
* Get around this by passing [chat history](https://python.langchain.com/docs/concepts/chat_history/) 
  * Record of conversation between the user & chat model
  * Used to **maintain context & state throughout conversation**
* Sequence of [`messages`](https://python.langchain.com/docs/concepts/messages/)
  * Each has associated specific [`role`](https://python.langchain.com/docs/concepts/messages/#role)
    * `"user"`
    * `"assistant"`
    * `"system"`

In [16]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 33, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'stop', 'logprobs': None}, id='run-352e12c0-c655-4d13-a9dc-c10b71e19879-0', usage_metadata={'input_tokens': 33, 'output_tokens': 15, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Message Persistence

* [`LangGraph`](https://langchain-ai.github.io/langgraph/) implements **built-in persistence layer**  → ideal for chat applications with support **multiple conversational turns**

* Wrapping `ChatModel` in **minimal LangGraph application** → automatically persist message history, simplifying development of multi-turn applications

* LangGraph comes with simple **in-memory `checkpointer`**
  
* [**Persistence Documentation**](https://langchain-ai.github.io/langgraph/concepts/persistence/) for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).

> ℹ️ **API Reference**: [`MemorySaver`](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.MemorySaver)

 > ℹ️ **API Reference**: [`StateGraph`](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.state.StateGraph)

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

* Need to create `config` to pass to `runnable` 
  * contains information not part of input directly, but is still useful. 
  * In this case, we want to include a `thread_id`

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

* **Supports multiple conversation threads with single application**
* Common requirement: application has multiple users!

In [8]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


If you config a new `thread` conversation strarts fresh 🌿🌿🌿

In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. How can I assist you today?


But if we go back to `{"thread_id":"abc123"}` it remembers us

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. What would you like to talk about?


### ‼️‼️‼️‼ `thread_id` is how you support a chatbot having convo with many users